In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import *
from sklearn.preprocessing import *
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
train_data = pd.read_csv("")

In [3]:
train_data.head()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,2,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,2,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,2,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54


In [4]:
train_data.isnull().sum()

LABEL        0
FLUX.1       0
FLUX.2       0
FLUX.3       0
FLUX.4       0
            ..
FLUX.3193    0
FLUX.3194    0
FLUX.3195    0
FLUX.3196    0
FLUX.3197    0
Length: 3198, dtype: int64

In [5]:
train_data = train_data.drop(columns=['LABEL'])

In [6]:
X = train_data.iloc[:, :-1].values  # FLUX1부터 FLUX3196까지
y = train_data.iloc[:, -1].values   # FLUX3197
y

array([-39.54,  19.93, -96.67, ...,  79.43,  -2.55,  27.82])

In [7]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

X_scaled.shape,y_scaled.shape

((5087, 3196), (5087, 1))

In [8]:
def create_sequences(data, labels, seq_length):
    sequences = []
    seq_labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        seq_labels.append(labels[i+seq_length])
    return np.array(sequences), np.array(seq_labels)

In [9]:
# 시퀀스 길이 설정
seq_length = 25

In [10]:
# 시퀀스 데이터 생성
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, seq_length)

In [11]:
y_sequences.shape

(5062, 1)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

In [13]:
# 텐서로 변환 및 GPU로 이동
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

In [14]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc_1 = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)  # 드롭아웃 비율 조정
        self.batch_norm = nn.BatchNorm1d(128)  # 배치 정규화 레이어 추가

        self.relu = nn.ReLU()

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.batch_norm(out)  # 배치 정규화 적용
        out = self.relu(out)
        out = self.dropout(out)  # 드롭아웃 적용
        out = self.fc(out)
        return out

In [15]:
# 하이퍼파라미터 설정
input_dim = X_train_tensor.shape[2]
hidden_dim = 64
layer_dim = 1
output_dim = X_train_tensor.shape[2]  # 예측할 값의 차원

In [16]:
# 모델 초기화 및 GPU로 이동
model = LSTM(output_dim, input_dim, hidden_dim, layer_dim, seq_length).to(device)

In [17]:
# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
# 모델 학습
num_epochs = 1000

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    # 검증 데이터에 대한 성능 평가
    if epoch % 10 == 0:
        model.eval()
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

C:\Users\user\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([4049, 1])) that is different to the input size (torch.Size([4049, 3196])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\user\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1013, 1])) that is different to the input size (torch.Size([1013, 3196])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [0/1000], Loss: 0.9265, Val Loss: 2.2711
Epoch [10/1000], Loss: 0.7467, Val Loss: 2.2701
Epoch [20/1000], Loss: 0.7073, Val Loss: 2.2696
Epoch [30/1000], Loss: 0.6921, Val Loss: 2.2694
Epoch [40/1000], Loss: 0.6668, Val Loss: 2.2709
Epoch [50/1000], Loss: 0.5669, Val Loss: 2.2740
Epoch [60/1000], Loss: 0.3391, Val Loss: 2.2751
Epoch [70/1000], Loss: 0.2733, Val Loss: 2.2836
Epoch [80/1000], Loss: 0.2469, Val Loss: 2.3250
Epoch [90/1000], Loss: 0.2319, Val Loss: 2.3675
Epoch [100/1000], Loss: 0.2278, Val Loss: 2.3217
Epoch [110/1000], Loss: 0.2158, Val Loss: 2.4020
Epoch [120/1000], Loss: 0.2345, Val Loss: 2.3686
Epoch [130/1000], Loss: 0.2069, Val Loss: 2.3635
Epoch [140/1000], Loss: 0.1654, Val Loss: 2.3705
Epoch [150/1000], Loss: 0.1821, Val Loss: 2.3801
Epoch [160/1000], Loss: 0.1188, Val Loss: 2.3305
Epoch [170/1000], Loss: 0.0984, Val Loss: 2.3759
Epoch [180/1000], Loss: 0.1030, Val Loss: 2.5034
Epoch [190/1000], Loss: 0.1003, Val Loss: 2.3573
Epoch [200/1000], Loss: 0.1080,